# Example for BCC Ni electronic band structure (non-magnetic state)
### *Conquest vs QEspresso*

In [ ]:
import os
import shutil
import subprocess
import matplotlib.pyplot as plt
import pickle

from scipy.integrate import cumulative_trapezoid, trapezoid

from distutils.spawn import find_executable
from numpy import amax, amin, size, array, trapz, floor, cumsum

from ase.calculators.conquest import Conquest
from ase.io.conquest import Conquest_orthorhombic_check
from ase.io.conquest import get_fermi_level
from ase.io.conquest import read_conquest

from ase.calculators.espresso import Espresso
from ase.io.espresso import grep_valence

from ase.units import Hartree
from ase.build import bulk
from ase.dft.bandgap import bandgap
from ase.dft.dos import DOS
from ase.visualize import view
from ase.spacegroup import get_spacegroup
from ase import Atoms

from cq_ase_external_lib import print_struct_data
from cq_ase_external_lib import get_gapwind

In [ ]:
%%bash 
ase --version

## Tools setup

- Check if visualisation tools are installed

In [ ]:
# Add exe name 
cmd_vis = {'xcrysden' : False}

for cmd, state in cmd_vis.items():
    if ( not shutil.which(cmd) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   
        cmd_vis[cmd] = True
        
# For MacOSX add the path of the name of the app
cmd_app = {'VESTA'      : '/Applications/VESTA/VESTA.app/Contents/MacOS/VESTA',
           'VMDLauncher': '/Applications/VMD_1.9.2.app/Contents/MacOS/VMDLauncher'}

for cmd, path in cmd_app.items():
    if ( not find_executable(path) ):
        print('{} not found'.format(cmd))
    else:
        print('{} found'.format(cmd))   

- Directory for storing calculation files

In [ ]:
working_directory = 'vs_qe_cq_example_Ni_FCC_bands'

# Test if `working_directory` exists ? If not create it
if ( not os.path.isdir(working_directory) ):
    os.makedirs(working_directory)

## Generate rocksalt cell from CIF file data

- Choose either primitive or conventional cell

In [ ]:
primitive = False

- Generate Ni FCC struture with $a=3.51$ Ang.

In [ ]:
if ( primitive ):
    # For primitive cell
    struct = bulk('Ni', crystalstructure='fcc', a=3.51, cubic=False)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # warning must be printed and 'struct' is modified to conventional cell
    
else:
    # For conventional cell
    struct = bulk('Ni', crystalstructure='fcc', a=3.51, cubic=True)
    # Conquest can only handle orthorhombic cells ; check
    struct = Conquest_orthorhombic_check(struct,verbose=False)
    # no warning...

- Extract and print main structural data from Atoms object (here `struct`)

In [ ]:
print_struct_data(struct,verbose=1)

- Space group ; Brillouin zone of space group [225](https://www.cryst.ehu.es/cgi-bin/cryst/programs/nph-kv-list?gnum=225&fig=fm3qmf&what=data)

In [ ]:
sg = get_spacegroup(struct)
print('Space group: {} ({}) '.format(sg.symbol,sg.no))

- Save input structure as VASP `POSCAR` file & `CIF` file (for checking purpose)

In [ ]:
struct.write(working_directory+'/input.vasp')
struct.write(working_directory+'/input.cif')

- run `VESTA` to check input (if possible) from `input.cif`

In [ ]:
if ( 'VESTA' in cmd_app ):
    subprocess.run([cmd_app['VESTA'], working_directory+'/'+'input.cif'])

## Band-path setup

- Lattice description

In [ ]:
latt  = struct.cell.get_bravais_lattice()
print(latt.description())
#print('lattice = ', latt)
#print('special k-point = ',list(latt.get_special_points()))

- Choose bandpath

In [ ]:
if ( primitive ):
    # For primitive cell
    path = 'GXWKGLUWLK'
else:
    # For conventional cell
    path = 'GXMGRX,MR'

- Setup kpath and plot

In [ ]:
nkpts = 100
kpath = struct.cell.bandpath(path, npoints=nkpts)
#latt.plot_bz(show=True,path=path)

#size1, size2 = kpath.kpts.shape
#print('k-point array shape =',size1,'x',size2)
#print('k-point coord list (fractional coord.)')
#print(kpath.kpts) 

## SCF and band structure calculation with Conquest

- Copy ASE `struct` object file

In [ ]:
cq_struct = struct

- Define Conquest environment variables

In [ ]:
os.environ['ASE_CONQUEST_COMMAND'] = 'mpirun -np 4 /Users/lioneltruflandier/CONQUEST-release-f-rework-output/src/Conquest'
os.environ['CQ_PP_PATH'] = '/Users/lioneltruflandier/Conquest-develop-outdated/pseudo-and-pao'
os.environ['CQ_GEN_BASIS_CMD'] = '/Users/lioneltruflandier/CONQUEST-release-develop/tools/BasisGeneration/MakeIonFiles'

- Setup Conquest basis set

In [ ]:
basis = {'Ni' : {'gen_basis'            : True,
                 'basis_size'           : 'large',
                 'pseudopotential_type' : 'hamann',
                 'xc'                   : 'PBE' },
        }

val_elec = {'Ni': 18}

- Given `struct`and the `val_elec`compute the total number of electrons

In [ ]:
# Get atom labels
atom_names = struct.get_chemical_symbols()

# Compute total valence electrons
tot_val = 0
for i in range(len(struct)):
    for key, val in val_elec.items():
        if ( key == atom_names[i] ):
            tot_val = tot_val + val

print('Number of valence electron in the unit cell = ', tot_val)

- Compute number(s) of occupied states

In [ ]:
n_occ = []
if ( tot_val % 2 == 0):
    # paired electrons
    n_occ.append(int(tot_val/2))
else:
    # unpaired electrons
    n_occ.append(int((tot_val-1)/2))
    n_occ.append(int((tot_val+1)/2))
    
print()
print('Number of occupied bands = ', n_occ)

- Setup data input specification (Ha unit)

In [ ]:
cutoff  =  90.0
kpoints = [6,6,6]
fxc     = 'PBE'
kT      = 0.005

#conquest_flags = {'General.DifferentFunctional' : 'True'}

conquest_flags = {'IO.WriteOutToASEFile' : True,
                  'IO.DumpChargeDensity' : True,
                  'Diag.SmearingType   ' : 0,
                  'Diag.kT'              : kT,
                  'minE.SCTolerance'     : 1e-8,
                  'General.NeutralAtom'  : False
                 }

- Setup calculation using Conquest as calculator

In [ ]:
cq_struct.calc = Conquest(directory = working_directory,
                    grid_cutoff     = cutoff,
                    self_consistent = True,
                    xc    = fxc,
                    basis = basis,
                    kpts  = kpoints,
                    nspin = 1,
                    **conquest_flags)

#cq_struct.calc = calc
print(cq_struct.calc)

- <span style='background:orange'>  Launch SCF calculation (be patient...) </span>

In [ ]:
dft_energy = cq_struct.get_potential_energy()

- Copy SCF calculation input/output files 

In [ ]:
subprocess.run(['cp', working_directory+'/'+'Conquest_input', 
                working_directory+'/'+'Conquest_input_scf'])

subprocess.run(['cp', working_directory+'/'+'Conquest_out', 
                working_directory+'/'+'Conquest_out_scf'])

- Extract results

In [ ]:
cq_struct.calc.read_results()

- Extract and print Fermi energy from SCF 

In [ ]:
cq_fermi_energy_scf = struct.calc.eFermi

print('Fermi SCF = {:12.4f} eV'.format(cq_fermi_energy_scf))
cq_fermi_energy = cq_fermi_energy_scf

- Setup Conquest band structure parameters

In [ ]:
changed_parameters = cq_struct.calc.set(kpts=kpath.kpts,self_consistent=False,**conquest_flags)

- ...or setup `kpoint_lines` in Conquest bands input

In [ ]:
#kpts_lines = {'path': path, 'npoints': nkpts, 'special_points' : '{'+path+'}'}
#changed_parameters = calc.set(kpts=kpts_lines,self_consistent=False,**conquest_flags)

- <span style='background:orange'> Run band structure calculation (be patient...) </span>



In [ ]:
cq_struct.calc.calculate(cq_struct)

- Extract Conquest band structure

In [ ]:
cq_bs = cq_struct.calc.band_structure()

- Setup energy reference for band structure plot

In [ ]:
cq_bs._reference = cq_fermi_energy

# Rescaled energies if needed : Fermi energy setup to 0.0 !
cq_bs._energies  = cq_bs._energies  - cq_fermi_energy
cq_bs._reference = cq_bs._reference - cq_fermi_energy

- Print Fermi energy and energy spectrum bounds

In [ ]:
print('Fermi = {:12.4f} eV'.format(cq_fermi_energy))
e_max = amax(cq_bs.energies) ; print('e_max = {:12.4f} eV'.format(e_max))
e_min = amin(cq_bs.energies) ; print('e_min = {:12.4f} eV'.format(e_min))

- Compute DOS ; Fermi energy must be given

In [ ]:
cq_dos = DOS(cq_struct.calc, width=0.1, fermi=[cq_fermi_energy], npts=2201)

In [ ]:
cq_d = cq_dos.get_dos() 
cq_e = cq_dos.get_energies() 

In [ ]:
%matplotlib inline
from numpy import shape

# Get the total number of states
nspin, nkpts, nstates = shape(cq_bs.energies)

# Choose energy window [e_min, e_max]
e_max =  25.0
e_min = -15.0

# Set ax0 and ax1 on 1 single figure
fig, (cq_ax0, cq_ax1) = plt.subplots(1,2)

# Plot DOS on ax1
cq_ax1.plot(cq_d, cq_e, color='black')

# Set plots limits
cq_ax1.set_ylim(e_min,e_max)
cq_ax1.set_xlim(0.0,n_occ[0])

# Plot horizontal and vertical lines and x-axis label
cq_ax1.axhline(cq_bs.reference, color='k', ls=':')
cq_ax1.axvline(n_occ[0], color='k', ls=':', linewidth=0.2)
cq_ax1.set_xlabel('state count')

# Plot band structure on ax0
cq_ax0.set_xlabel('$k$-path')
cq_bs.plot(ax=cq_ax0, emin=e_min, emax=e_max, show=False, color='black')


- Compute direct and indirect band gap

In [ ]:
#WARNING: accuracy issue from ASE 'bandgap' (not yet resolved)
#solution: add epsilon to bs._reference (fermi_energy)
eps = 1e-6
cq_gap_d, p1, p2 = bandgap(cq_struct.calc,efermi=cq_fermi_energy+eps,direct=True)
cq_gap_i, p1, p2 = bandgap(cq_struct.calc,efermi=cq_fermi_energy+eps,direct=False)

## SCF and band structure calculation with QEspresso

- Copy ASE `struct` object file

In [ ]:
qe_struct = struct

- Define QEspresso environment

In [ ]:
os.environ['ASE_ESPRESSO_COMMAND']= 'mpirun -np 4 /Users/lioneltruflandier/QEspresso/qe-7.1/bin/pw.x  < PREFIX.pwi > PREFIX.pwo'
os.environ['ESPRESSO_PSEUDO']     = '/Users/lioneltruflandier/QEspresso/pseudo_dojo'

- Define pseudopotential for Na and Cl and valence electrons

In [ ]:
pseudo_path      = os.environ['ESPRESSO_PSEUDO']
pseudopotentials = {'Ni': 'Ni.upf'}

val_elec = { }
for symbol, pseudo in pseudopotentials.items():
    print(symbol,pseudo,grep_valence(pseudo_path+'/'+pseudo))
    val_elec[symbol] = [grep_valence(pseudo_path+'/'+pseudo),1]
    
for key, val in val_elec.items():
    print(key,val)

- Given `struct`and the `val_elec`compute the total number of electrons

In [ ]:
# Get atom labels
atom_names = cq_struct.get_chemical_symbols()

# Compute total valence electrons
tot_val = 0
for i in range(len(struct)):
    for key, val in val_elec.items():
        if ( key == atom_names[i] ):
            tot_val = tot_val + val[0]

print('Number of valence electron in the unit cell = ', tot_val)

- Compute number(s) of occupied states

In [ ]:
n_occ = []
if ( tot_val % 2 == 0):
    # paired electrons
    n_occ.append(int(tot_val/2))
else:
    # unpaired electrons
    n_occ.append(int((tot_val-1)/2))
    n_occ.append(int((tot_val+1)/2))
    
print()
print('Number of occupied bands = ', n_occ)

- Define number of unoccupied states (make sens only for bands calculation)

In [ ]:
n_ratio = 1.0
n_virt  = [int(x*n_ratio) for x in n_occ]
print('Number of unoccupied bands = ', n_virt)
#n_virt =

# Compute total
n_tot = max(n_occ) + max(n_virt)
print('Total number of bands = ', n_tot)

- Setup data input specification (Ry unit)

In [ ]:
ecutwfc = 90.0
ecutrho = ecutwfc*6

occupation= 'smearing'
smearing  = 'fermi-dirac'
kT = 0.01 # Ry unit

input_data = {
        'system' : {
        'ecutwfc': ecutwfc,
        'ecutrho': ecutrho,
        'occupations' : occupation,
        'smearing'    : smearing,
        'degauss'     : kT}        
        }


- Setup calculation using Quantum-Espresso as calculator

In [ ]:
qe_struct.calc = Espresso(directory = working_directory,
                          pseudopotentials = pseudopotentials,
                          tstress = True, 
                          tprnfor = True, 
                          kpts    = kpoints,
                          input_data = input_data)

-  <span style='background:orange'> Launch electronic structure calculation (be patient...) </span>

In [ ]:
qe_dft_energy = qe_struct.get_potential_energy()

- Copy SCF calculation input/output files 

In [ ]:
subprocess.run(['cp', working_directory+'/'+'espresso.pwi', 
                working_directory+'/'+'espresso.pwi_scf'])

subprocess.run(['cp', working_directory+'/'+'espresso.pwo', 
                working_directory+'/'+'espresso.pwo_scf'])

- Extract Fermi energy from SCF 

In [ ]:
qe_fermi_energy_scf = struct.calc.get_fermi_level()

- Setup QEspresso band structure parameters

In [ ]:
occupation= 'smearing'
smearing  = 'fermi-dirac'
kT = 0.01 # Ry unit

input_data.update( {'control':
                   {'calculation' :'bands',
                    'restart_mode':'restart',
                    'verbosity'   :'high'}} 
                 ) 
                   
input_data.update( {'system':{'nbnd'   : n_tot,
                              'ecutwfc': ecutwfc,
                              'ecutrho': ecutrho,
                              'occupations' : occupation,
                              'smearing'    : smearing,
                              'degauss'     : kT}}
                 )
input_data.update( {'electrons': {'diago_full_acc' : True }} )

In [ ]:
changed_parameters = qe_struct.calc.set(kpts=kpath,input_data=input_data)

-  <span style='background:orange'> Run band structure calculation (be patient...) </span>

In [ ]:
qe_struct.calc.calculate(qe_struct)

- Extract QEspresso band structure

In [ ]:
qe_bs = qe_struct.calc.band_structure()

- Choose wich Fermi level to use for bands and DOS post-processing 

> you have choice between `fermi_energy_scf` and `fermi_energy_bands`. For insulator, the first one is evaluated to be within the gap. The second one is setup at the highest occupied energy level.

In [ ]:
print('fermi SCF   = {:12.4f} eV'.format(qe_fermi_energy_scf))
qe_fermi_energy = qe_fermi_energy_scf


- Setup energy reference for band structure plot

In [ ]:
qe_bs._reference = qe_fermi_energy

# Rescaled energies if needed : Fermi energy setup to 0.0 !
qe_bs._energies  = qe_bs._energies  - qe_fermi_energy
qe_bs._reference = qe_bs._reference - qe_fermi_energy

- Compute DOS ; Fermi energy must be given

In [ ]:
qe_dos = DOS(qe_struct.calc, width=0.1, fermi=[qe_fermi_energy],npts=1201)
qe_d = qe_dos.get_dos() / 2  # /2 to correct for ASE spin unpolarized
qe_e = qe_dos.get_energies() 

- Plot and print `QEspresso` and `Conquest` band structure and DOS

In [ ]:
%matplotlib notebook

# Choose energy window [e_min, e_max]
e_max =  25.0
e_min = -10.0

fig2, (qe_ax0, qe_ax1) = plt.subplots(1,2)

# Plot QEspresso DOS on qe_ax1
qe_ax1.plot(qe_d, qe_e, color='black')

# Plot Conquest DOS on qe_ax1
qe_ax1.plot(cq_d, cq_e, color='blue' )
    
# Set labels and stuff on DOS plot
qe_ax1.set_xlabel('state count')

# Set plots x- and y- limits
qe_ax1.set_xlim(0.0,n_occ[0])
qe_ax1.set_ylim(e_min,e_max)

# Plot horizontal and vertical lines and x-axis labels
qe_ax1.axhline(qe_bs.reference, color='k', ls=':')
qe_ax1.axvline(n_occ[0], color='k', ls=':', linewidth=0.2)
qe_ax1.set_xlabel('state count')

# Plot band strutures on ax0
qe_ax0.set_xlabel('$k$-path')
qe_bs.plot(ax=qe_ax0,emin=e_min, emax=e_max, show=True,color='black')
cq_bs.plot(ax=qe_ax0,emin=e_min, emax=e_max, show=True,color='blue' )

- Save figure as `bands.png` in `working_directory`

In [ ]:
# Save band strutures
fig2.savefig( working_directory+'/'+'bands.png', 
             dpi = 300, format='png', transparent=True, bbox_inches ='tight')